In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from DLtools import  prep_data,TSmodel
np.random.seed(24)

In [3]:
from sklearn import linear_model
import statsmodels.api as sm

In [4]:
rain = 'data/instant_data/rain.csv'
water = 'data/instant_data/water.csv'

bkk = 'CPY015_w' # target station

# select from 2013 due to missing data issue
start = '2019-01-01'
stop = '2020-01-01'
rw = prep_data.instant_df(rain_csv=rain,water_csv=water,start=start,stop=stop)

df = rw.df
col_w,col_r = rw.only_related()


In [5]:
def report_missing_by_station(df,col):
    nan_ratio = 0.15
    print(col,'\n')
    print("***From current station {} list below, find and discard station that too much missingdata \n".format(len(col)))
    print("missing data over {}%, as below".format(nan_ratio*100))
    for _col in df:   
        num_nan = df[_col].isnull().sum()
        num_total = df.shape[0]
        if num_nan/num_total > nan_ratio:
            try:
                col.remove(_col)
            except:
                pass
        else:
            print("REMAIN Station name {}, ,missing......{} (..{:.2f}%)".format(_col,num_nan,num_nan*100/num_total))
    print("total data points",num_total)
    print("remain .......{}...station".format(len(col)))
    print(col)
    return col

In [6]:
report_missing_by_station(df,col_w)
report_missing_by_station(df,col_r)

['CHM004_w', 'BKK018_w', 'BKK004_w', 'NAN011_w', 'NAN003_w', 'CPY015_w', 'DIV003_w', 'NAN013_w', 'NAN002_w', 'NAN006_w', 'NAN009_w', 'THA001_w', 'CPY002_w', 'GLF001_w', 'NAN010_w', 'TBW008_w', 'CPY005_w', 'VLGE12_w', 'CPY012_w', 'CPY006_w', 'CPY017_w', 'CPY013_w', 'PIN003_w', 'LBI001_w', 'CPY007_w', 'PIN004_w', 'THA002_w', 'PIN005_w', 'BKK019_w', 'CPY004_w', 'DIV005_w', 'NAN014_w', 'CPY014_w', 'LBI002_w', 'TB0004_w', 'HDA004_w', 'BKK007_w', 'NAN005_w', 'CPY016_w', 'NAN004_w', 'BKK013_w', 'KWN002_w', 'NAN012_w', 'CPY008_w', 'TBW014_w', 'CHM001_w', 'TB0001_w', 'CPY010_w', 'CPY009_w', 'BKK003_w', 'TB0006_w', 'HDA005_w', 'NAN008_w', 'NAN001_w', 'BKK001_w', 'NAN007_w', 'CPY001_w', 'BKK020_w', 'CHM002_w', 'CHM006_w', 'PIN001_w', 'CHM005_w', 'CPY003_w', 'BKK002_w', 'BKK008_w', 'PIN006_w', 'KWN001_w', 'BKK005_w', 'TB0002_w', 'PIN002_w', 'BKK021_w', 'CPY011_w'] 

***From current station 72 list below, find and discard station that too much missingdata 

missing data over 15.0%, as below
REMAIN 

[]

In [7]:
df.shape

(53070, 155)

In [14]:

X = df.resample('d').mean()
X = X.interpolate(method='pad', limit=300)
X = X.fillna(0)
y = df['CPY015_w'].resample('d').mean()

In [16]:
regr = linear_model.LinearRegression()
regr.fit(X, y)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

Intercept: 
 -2.320588166071502e-12
Coefficients: 
 [ 6.39627468e-15 -1.88534084e-14  1.52675182e-14  1.27671311e-14
  3.81639165e-15  1.00000000e+00 -1.88564442e-15  2.84147705e-15
 -7.89993071e-15 -3.71230824e-15  9.13158438e-15  4.63656891e-14
  6.41847686e-16  4.25527669e-14 -8.18789481e-16 -1.05408846e-15
  4.44089210e-16  8.32667268e-17 -7.86697096e-15  1.04777298e-15
 -4.49640325e-15 -2.91780489e-15 -9.54965274e-15  4.12473875e-14
 -1.84088855e-14 -4.99600361e-16 -2.69922973e-15 -2.53269627e-15
  1.37485509e-14 -4.46795378e-14  1.24900090e-16 -3.59261232e-15
 -1.29783337e-14 -9.40914013e-15  2.88657986e-15  1.63202785e-14
  8.04131067e-14 -9.36750677e-17 -7.48099499e-15  1.42837131e-14
  6.21031004e-15  2.52575738e-15  4.96130914e-16  2.80851731e-14
  4.16333634e-17  6.24500451e-16  4.66814087e-15 -1.06613936e-14
 -2.32973363e-15  3.35929201e-15  2.45626002e-14 -1.63757896e-14
 -4.92661467e-15 -9.27036226e-15  8.81911731e-15  2.77555756e-17
  8.83321194e-15 -5.12263842e-15 -1.27

In [36]:
model = sm.OLS(y, X)
result = model.fit()
print(result.summary())

                                 OLS Regression Results                                
Dep. Variable:               CPY015_w   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          3.925e+24
Date:                Wed, 28 Oct 2020   Prob (F-statistic):                        0.00
Time:                        23:43:45   Log-Likelihood:                          9935.8
No. Observations:                 366   AIC:                                 -1.957e+04
Df Residuals:                     214   BIC:                                 -1.897e+04
Df Model:                         152                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [39]:
predictions = result.predict(X) 
len(predictions)
 

366

In [43]:
from sklearn.metrics import mean_squared_error
error = mean_squared_error(y, predictions)
print('Test MSE: {:.3f}'.format(error*100))


Test MSE: 0.000
